In [ ]:
!pip install faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.1 MB/s eta 0:00:00


In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from google.colab import drive
drive.mount('/content/drive')
dir_path = "/content/drive/Shareddrives/Rays-Proyecto Integrador/Python-Firebase/bitesight-858b3-firebase-adminsdk-rikzy-3c4a947c87.json"
cred = credentials.Certificate(dir_path)
firebase_admin.initialize_app(cred)

db = firestore.client()

Mounted at /content/drive


In [ ]:
from faker import Faker
import random

fake = Faker()

def generate_user():
    user = {}
    if random.choice([True, False]):
        user["name"] = fake.name_male()
        user["gender"] = "male"
    else:
        user["name"] = fake.name_female()
        user["gender"] = "female"
    names = user["name"].split()
    email_username = ".".join(names).lower()
    user["email"] = f"{email_username}@example.com"
    user["birthday"] = fake.date_of_birth().strftime('%Y/%m/%d')

    mu, sigma = 2.5, 0.5
    pricing = round(random.gauss(mu, sigma))
    user["pricing"] = max(1, min(pricing, 4))

    relationships = {
        'Restaurante': ['Restaurante familiar', 'Restaurante de comida para llevar'],
        'Bar restaurante': ['Pub restaurante', 'Cervecería'],
        'Marisquería': ['Restaurante de marisco donburi', 'Restaurante de fish & chips'],
        'Restaurante italiano': ['Pizzería'],
        'Restaurante asiático': ['Restaurante cantonés', 'Restaurante chino', 'Restaurante japonés', 'Restaurante de sushi'],
        'Restaurante americano': ['Hamburguesería', 'Restaurante de comida tradicional americana', 'Restaurante de alas de pollo','Restaurante tex-mex'],
        'Restaurante mexicano': ['Restaurante de comida casera', 'Taquería'],
        'Delicatessen': [],
        'Restaurante de comida saludable': [],
        'Restaurante especializado en carnes': ['Parrilla', 'Restaurante especializado en filetes'],
        'Restaurante étnico': ['Restaurante cubano', 'Restaurante brasileño', 'Restaurante argentino', 'Restaurante alemán']
}
    

    user["likes"] = {}
    for category in relationships:
        user["likes"][category] = fake.boolean(chance_of_getting_true=60)
        for related_category in relationships[category]:
            if user["likes"][category]:
                user["likes"][related_category] = fake.boolean(chance_of_getting_true=90)
            else:
                user["likes"][related_category] = fake.boolean(chance_of_getting_true=60)

    return user

In [ ]:
user = generate_user()
user

{'name': 'James Gutierrez',
 'gender': 'male',
 'email': 'james.gutierrez@example.com',
 'birthday': '1986/09/17',
 'pricing': 2,
 'likes': {'American': True,
  'Asian': True,
  'Brewery': True,
  'Delicatessen': False,
  'Ethnic': False,
  'Healthy': True,
  'Italian': True,
  'Mexican': True,
  'Seafood': False,
  'Steakhouse': True,
  'Takeout': False,
  'Wings': True}}

In [ ]:
users_ref = db.collection('users')

In [ ]:
# for i in range(250):
#     user = generate_user()
#     users_ref.add(user)

In [ ]:
import random
from datetime import datetime
from typing import List, Dict

In [ ]:
category_mapping = {
    'American': ['Restaurante americano', 'Hamburguesería', 'Restaurante de comida tradicional americana'],
    'Asian': ['Restaurante asiático', 'Restaurante cantonés', 'Restaurante chino', 'Restaurante japonés', 'Restaurante de sushi'],
    'Brewery': ['Cervecería'],
    'Delicatessen': ['Delicatessen'],
    'Ethnic': ['Restaurante étnico', 'Restaurante cubano', 'Restaurante brasileño', 'Restaurante argentino', 'Restaurante alemán'],
    'Healthy': ['Restaurante de comida saludable'],
    'Italian': ['Restaurante italiano', 'Pizzería'],
    'Mexican': ['Restaurante mexicano', 'Restaurante tex-mex', 'Taquería'],
    'Seafood': ['Marisquería', 'Restaurante de marisco donburi', 'Restaurante de fish & chips'],
    'Steakhouse': ['Restaurante especializado en carnes', 'Parrilla', 'Restaurante especializado en filetes'],
    'Takeout': ['Restaurante de comida para llevar'],
    'Wings': ['Restaurante de alas de pollo']
}

In [11]:
def generate_ratings() -> List[Dict]:
    ratings = []
    users_ref = db.collection('users')
    users = users_ref.get()
    restaurants_ref = db.collection('restaurants')
    restaurants = restaurants_ref.get()
    for user in users:
        user_data = user.to_dict()
        if 'likes' in user_data:
            for i in range(random.randint(2, 3)):
                restaurant = random.choice(restaurants)
                restaurant_data = restaurant.to_dict()
                if any(user_data['likes'].get(category) for category in category_mapping.keys() if any(r_category in category_mapping[category] for r_category in restaurant_data['categories'])):
                    rating = random.randint(3, 5)
                else:
                    rating = random.randint(1, 5)
                rating_doc_ref = db.collection('ratings').document()
                rating_doc_ref.set({
                    'user_id': user.id,
                    'restaurant_id': restaurant.id,
                    'rating': rating,
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                })
    return ratings

In [12]:
# generate_ratings()

[]